# `Label & One Hot Encoding: with SkLearn`

# <font color = red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ Create One Hot & Label Encoding with `skLearn`
    + When to use one over the other and thoughts
+ Learn how to endcode and decode your variables
    + Use single and multiple columns for encoding

`Original Data`:

| Country  	| Device 	| Version 	|
|----------	|--------	|---------	|
| Serbia   	| iPad   	| 10_3_4  	|
| Qatar    	| iPhone 	| 9_3_5   	|
| Cambodia 	| iPad   	| 12_4    	|
| Fiji     	| iPad   	| 7_1_2   	|


`Data Encoded`:  **End Result**
 
| Country  	| Device 	| Version 	| Country_LabEnc 	| Version_LabEnc 	| Device_OneHotEnc 	|
|----------	|--------	|---------	|----------------	|----------------	|------------------	|
| Serbia   	| iPad   	| 10_3_4  	| 164            	| 1              	| 0                	|
| Qatar    	| iPhone 	| 9_3_5   	| 127            	| 8              	| 1                	|
| Cambodia 	| iPad   	| 12_4    	| 29             	| 2              	| 0                	|
| Fiji     	| iPad   	| 7_1_2   	| 60             	| 7              	| 0                	|

In [147]:
import pandas as pd # DF 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder  # Encoding
import faker # fake data generator
from collections import defaultdict

In [121]:
# Generate Random Countries:

g=faker.Faker()

countries=[]
for i in range(500):
    countries.append(g.country())
    


In [125]:
# Fake Devices and Device Versions:

import re
Os_=[]
for _ in range(500):
#     print(g.ios_platform_token())
    p=re.sub('\;','',g.ios_platform_token())
    Os_.append([p.split(' ')[0],p.split(' ')[4]])


500

In [143]:
# Create Dataframe:
df=pd.DataFrame(Os_,countries,columns=['Device','Version'])

# Remove 1st columns set as index which is your countries
df.reset_index(level=0, inplace=True)

# Rename the new column from index -> country
df=df.rename(columns={'index':'Country'})

In [145]:
df.head()

,Country,Device,Version
0,Serbia,iPad,10_3_4
1,Qatar,iPhone,9_3_5
2,Cambodia,iPad,12_4
3,Fiji,iPad,7_1_2
4,Singapore,iPhone,5_1_1


# Label Encoding:

+ First example, 1 column at a time
+ Second, multiple coulmns at once

`-------------------------------`

`Original Data`:

| Country  	| Device 	| Version 	|
|----------	|--------	|---------	|
| Serbia   	| iPad   	| 10_3_4  	|
| Qatar    	| iPhone 	| 9_3_5   	|
| Cambodia 	| iPad   	| 12_4    	|
| Fiji     	| iPad   	| 7_1_2   	|

`After encoding`: [*'Country','Version'*]

| Country  	| Device 	| Version 	| Country_LabEnc 	| Version_LabEnc 	|
|----------	|--------	|---------	|----------------	|----------------	|
| Serbia   	| iPad   	| 10_3_4  	| 164            	| 1              	|
| Qatar    	| iPhone 	| 9_3_5   	| 127            	| 8              	|
| Cambodia 	| iPad   	| 12_4    	| 29             	| 2              	|
| Fiji     	| iPad   	| 7_1_2   	| 60             	| 7              	|

In [153]:
# create label encoder
labelencoder = LabelEncoder()

# label encoding works by alphabetical order
df['Country_LabEnc']= labelencoder.fit_transform(df['Country'])
df.head()

df['Version_LabEnc']=labelencoder.fit_transform(df['Version'])
df.head()

,Country,Device,Version,Country_LabEnc,Version_LabEnc
0,Serbia,iPad,10_3_4,164,1
1,Qatar,iPhone,9_3_5,147,8
2,Cambodia,iPad,12_4,29,2
3,Fiji,iPad,7_1_2,60,7
4,Singapore,iPhone,5_1_1,167,5


# Lets Investigate the Version Label encoding:

In [191]:
print('Version Labels:',df['Version_LabEnc'].unique())
print('Version:',df['Version'].unique())

o=list(zip(df['Version_LabEnc'].unique(),df['Version'].unique()))

sorted(o, key=lambda x: x[0])

Version Labels: [1 8 2 7 5 3 4 0 6 9]
Version: ['10_3_4' '9_3_5' '12_4' '7_1_2' '5_1_1' '3_1_3' '4_2_1' '10_3_3' '6_1_6'
 '9_3_6']


[(0, '10_3_3'),
 (1, '10_3_4'),
 (2, '12_4'),
 (3, '3_1_3'),
 (4, '4_2_1'),
 (5, '5_1_1'),
 (6, '6_1_6'),
 (7, '7_1_2'),
 (8, '9_3_5'),
 (9, '9_3_6')]

# Reverse/Inverse Label Encoding:

In [195]:
print(labelencoder.fit_transform(df['Version'])[:5])

labelencoder.inverse_transform(labelencoder.fit_transform(df['Version']))[:5]

[1 8 2 7 5]


array(['10_3_4', '9_3_5', '12_4', '7_1_2', '5_1_1'], dtype=object)

# <font color=red>Consideration</font>:

+ If we are looking at this as `not a label but instead as ordinal data`; then what?
    + Well, we would have order to our data and need to reaccess! This appears to be the case; but it all depends how we want to analyze our data.
    
+ If we have skLearn take in our data it will act upon the order of which it is read in. We do have a problem and will need to adjust to make it assign our data in a specific order. 
    + There are times that even while switching encodings, we can have issues still and may need to further process as One Hot. Just bare in mind this when you are trying to figure out how to handle your data. 

In [281]:
# If we are to consider the versions let's first evaluate something:

print(10_3_1>10_3_2)
print(10.3>10.2)
print('Unexpected Result:','10_3'>'9_2')
print('Unexpected Result:','10_3_1'>'10_3_2')

'''
We need to evaluate the string as a literal value because it is ranking as 1 >9 
which is not what we want. The string will look at each number as they appear and do 
element wise comparison.
''' 

False
True
Unexpected Result: False
Unexpected Result: False


'\nWe need to evaluate the string as a literal value because it is ranking as 1 >9 \nwhich is not what we want. The string will look at each number as they appear and do \nelement wise comparison.\n'

In [286]:
# Comparing strings of numerical data: by ranking

int('10_3_1')>int('10_3_2')
int('10_2_1')>int('9_2_1')

True

# `How Should We Proceed?`

+ Brief explaination and code at end. Let's get into One Hot Encoding first

`--------------------------`

# One Hot Encoding:

In [184]:
# Create OneHot Encoder:
OneHt_enc = OneHotEncoder(handle_unknown='ignore')

# Create One hot as an array based on number of entries to eval: 
oneh_arry=pd.DataFrame(OneHt_enc.fit_transform(df[['Device']]).toarray())

# Join DF on key vals
df.join(oneh_arry).head()

,Country,Device,Version,Country_LabEnc,Version_LabEnc,0,1
0,Serbia,iPad,10_3_4,164,1,1.0,0.0
1,Qatar,iPhone,9_3_5,147,8,0.0,1.0
2,Cambodia,iPad,12_4,29,2,1.0,0.0
3,Fiji,iPad,7_1_2,60,7,1.0,0.0
4,Singapore,iPhone,5_1_1,167,5,0.0,1.0


# Inverse One Hot Encoding:

In [190]:
print(OneHt_enc.fit_transform(df[['Device']]).toarray()[:5])


encode_=OneHt_enc.fit_transform(df[['Device']])

# Transoform back to original Devices:
OneHt_enc.inverse_transform(encode_)[:5]

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


array([['iPad'],
       ['iPhone'],
       ['iPad'],
       ['iPad'],
       ['iPhone']], dtype=object)

`----------------------------------------`

# Encode Multiple Columns at Once:

In [251]:
df_=df.iloc[:,:3]

# Encoding:
enc_all_cols=df_.iloc[:,:3].apply(LabelEncoder().fit_transform)


#Encoding: But, columns have repeating names
df_w_enc=pd.concat([df.iloc[:,:3],enc_all_cols],axis=1).head()

# Rename New Encoded Columns:
df_w_enc.columns.values[3:]=['Country_LabEnc','Device_OneHt','Version_LabEnc']

df_w_enc

,Country,Device,Version,Country_LabEnc,Device_OneHt,Version_LabEnc
0,Serbia,iPad,10_3_4,164,0,1
1,Qatar,iPhone,9_3_5,147,1,8
2,Cambodia,iPad,12_4,29,0,2
3,Fiji,iPad,7_1_2,60,0,7
4,Singapore,iPhone,5_1_1,167,1,5


# Example 02: Multiple Columns Encoded

In [287]:
# Encode:

from collections import defaultdict
d = defaultdict(LabelEncoder)

fit = df_.apply(lambda x: d[x.name].fit_transform(x))
fit.head()

,Country,Device,Version
0,164,0,1
1,147,1,8
2,29,0,2
3,60,0,7
4,167,1,5


In [284]:
# Inverse the encoded (AKA decode)
fit.apply(lambda x: d[x.name].inverse_transform(x)).head()

,Country,Device,Version
0,Serbia,iPad,10_3_4
1,Qatar,iPhone,9_3_5
2,Cambodia,iPad,12_4
3,Fiji,iPad,7_1_2
4,Singapore,iPhone,5_1_1


# Lastly, How to deal with Ordinal Data:

+ If we are to consider the `Version` column to have some ranking or order: then we need to change it. Since we are dealing with strings of numbers separated by underscores we need to do a variation to the code.

In [303]:
# labelencoder.fit_transform(df['Version'])
k=[]
kk=[]
for i in df['Version']:
    k.append(int(i))
    u=labelencoder.fit_transform(k)
    uu=labelencoder.fit_transform(df['Version'])
print(u[:5])
print(df['Version'][:5])

print(int('12_4')>int('10_3_4'))
print(uu[:5])

[9 6 0 5 3]
0    10_3_4
1     9_3_5
2      12_4
3     7_1_2
4     5_1_1
Name: Version, dtype: object
False
[1 8 2 7 5]


In [324]:
print('Almost: but not quite',float('10_3_4'),float('12_4'))
print('Need to Refine________________')

num_lit_conv=[]

# re.findall(r'[]',)
for i in df['Version']:
    num_lit_conv.append(float(i.replace('_','.',1).replace('_','')))
num_lit_conv

print('Correct Format to Order:',sorted(set(num_lit_conv)))

Almost: but not quite 1034.0 124.0
Need to Refine________________
Correct Format to Order: [3.13, 4.21, 5.11, 6.16, 7.12, 9.35, 9.36, 10.33, 10.34, 12.4]


# `Therefore`: If we wanted to refind this (literal numeric)

+ we would have to do some formatting in order to correctly *map* as `Ordinal`

# <font color=red>`Future Work: further encoding examples with memory usage considerations` Look for this video soon</font>

In [344]:
print(labelencoder.fit_transform(num_lit_conv)[:7])
print(num_lit_conv[:7])
import numpy as np

np.unique(labelencoder.fit_transform(num_lit_conv))

# zip list to compare label and value:
lst_enc_version=list(zip(labelencoder.fit_transform(num_lit_conv),num_lit_conv))

# Correctly, labeled: and checking as a unique set to verify
set(sorted(lst_enc_version, key=lambda x: x[0]))

[8 5 9 4 2 0 1]
[10.34, 9.35, 12.4, 7.12, 5.11, 3.13, 4.21]


{(0, 3.13),
 (1, 4.21),
 (2, 5.11),
 (3, 6.16),
 (4, 7.12),
 (5, 9.35),
 (6, 9.36),
 (7, 10.33),
 (8, 10.34),
 (9, 12.4)}

In [349]:
# Bring it all together now:

correct_lab_enc_ord_vers=labelencoder.fit_transform(num_lit_conv)

# New Labels for Version 
df['Version_LabEnc_Fin']=correct_lab_enc_ord_vers

# Converted String -> float for encoding as Ordinal
df['Converted_litNum']=num_lit_conv

df.head()

,Country,Device,Version,Country_LabEnc,Version_LabEnc,Version_LabEnc_Fin,Converted_litNum
0,Serbia,iPad,10_3_4,164,1,8,10.34
1,Qatar,iPhone,9_3_5,147,8,5,9.35
2,Cambodia,iPad,12_4,29,2,9,12.40
3,Fiji,iPad,7_1_2,60,7,4,7.12
4,Singapore,iPhone,5_1_1,167,5,2,5.11


# <font color =red>LIKE</font>, Share &

# <font color=red>SUB</font>scribe

`------------------------`

# Citations & Help:

# ◔̯◔

https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

https://stackoverflow.com/questions/38101009/changing-multiple-column-names-but-not-all-of-them-pandas-python

https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/

https://medium.com/analytics-vidhya/types-of-categorical-data-encoding-schemes-a5bbeb4ba02b

https://towardsdatascience.com/categorical-encoding-techniques-93ebd18e1f24